# LLVM Prompt


In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import json
import csv
load_dotenv()

In [ ]:
openai_client = OpenAI(api_key=os.getenv("RUNPOD_TOKEN"), base_url=os.getenv('RUNPOND_CHATBOT_URL'))
model_name = os.getenv('MODEL_NAME')

In [ ]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": "How old is Samuel L Jackson"}
    ],
    temperature=0.0,
    top_p=0.8,
    max_tokens=2000
)

In [1]:
def get_chat_response(client, model_name, messages, temp=0):
    prompt_messages = []
    for message in prompt_messages:
        prompt_messages.append({"role": message["role"], "content": message["content"]})
        
    response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": "How old is Samuel L Jackson"}
    ],
    temperature=temp,
    top_p=0.8,
    max_tokens=2000
    ).choices[0].message.content
    
    return response

# LLM Response 

In [ ]:
messages = [{"role": "system", "content": "How old is Samuel L Jackson"}]
response = get_chat_response(openai_client, model_name, messages)

## Structured Outputs (JSON) and CSV Files (CSV)


In [ ]:
system_prompt="""

You are a helpful assistant that answers questions about  film stars.

Your output should be in a structured json format. You are allowed to write data in a JSON object:
[
    {
        "icon": "The celebrity icon that you will get the information about",
        "topic": "This information about the celebrity icon"
    }
]
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": "How old is Samuel L Jackson"})
response = get_chat_response(openai_client, model_name, messages)


try:
    json_response = json.loads(response) # JSON Formatting
    print("JSON Response:", json_response)
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    
csv_response = []

if csv_response:
    csv_filename = "celebrity_info.csv"
    with open(csv_filename, mode="w", newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        #header row
        writer.writerow(['icon', 'topic'])
        #data rows
        writer.writerows(csv_response)
        
    print(f"CSV File '{csv_filename}' has been successfully created.")
    
else: 
    print("No Data received: CSV unable to be written ")

# Input Structuring


In [ ]:
user_input = """
    Get me info about famous film stars:
    
    ```
        1. Samuel L Jackson
        2. Chris Rock
    ```
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": user_input})
response = get_chat_response(openai_client, model_name, messages)

json_response = json.loads(response)

# Chain of Thought 


In [ ]:
#Give the model time to think
# Giving the LLM more info makes the values more accurate

user_prompt = """
    Calculate the result of this question: 476*3/7000
    
    
    Your outupt should in a structured json format, exactyl as the one below.
    You cannot write anything other than the json object:
    
    {
        steps: This is where you solve the equation set by set following the BIDMAS. You need to show your working out and calcuate each step leading to the final result.
        result: The Final answer of the result of the equation above
    }
"""

messages = [{"role": "user", "content":user_prompt}]
response = get_chat_response(openai_client, model_name, messages)

# RAG (Retrival Augmented Generated) 

In [ ]:
user_prompt = """
What is newest features in a iphone 16
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chat_response(openai_client, model_name, messages)
print(response)

In [ ]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

user_prompt = f""" 
{iphone_16}

What's new in the iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chat_response(openai_client,model_name,messages)
print(response)